# Model 1: 6 Layers, 3 Conv and then 3 DeConv (Simple Implementation)

## Imports

In [ ]:
import os
from random import shuffle
from math import floor
from PIL import Image
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import load_model

## Set up paths, PLEASE put your path into the os.chdir() or this will not work

In [ ]:
#SET UP HERE CHANGE THE VALUE TO YOUR FILE DIRECTORY
os.chdir('C:/Users/YOUR_USER/Documents/Colourising_Photos/')

#Setting up path for data
pathColor = os.getcwd()+"/Data/colour/"
pathBW = os.getcwd()+"/Data/black/"

## Create model 1

In [ ]:
inputs = Input(name = 'input',shape = (256,256,1))
x = inputs
index = 0
w, h = (256, 256)

layer_filters = [32,64,128];

for current in layer_filters:
    x = Conv2D(filters = current, kernel_size = 3, activation='relu', padding='same', strides= 2)(x)
    x = Dropout(0.2)(x)

shape = int_shape(x) 
x = Flatten()(x)
x = Dense(256)(x)

x = Dense(shape[1]*shape[2]*shape[3])(x)
x = Reshape((shape[1],shape[2],shape[3]))(x)

index = 2
for current in layer_filters[::-1]:
    x = Conv2DTranspose(filters = current, kernel_size = 3,  activation='relu', padding="same", strides=2)(x)
    x = Dropout(0.2)(x)
    index += -1

outputs = Conv2DTranspose(filters=3, kernel_size = 3, activation='sigmoid',padding='same')(x)


model = Model(inputs, outputs, name='model')
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

## Load datas

In [ ]:
Xtrain = []
Ytrain = []
Xtest = []
Ytest = []
file = open(os.getcwd()+'/Data/train.txt')
trainFile = file.readlines()
file.close()


for f in trainFile:
    Ytrain.append( np.load(pathColor + f[:-1]))
    Xtrain.append( np.load(pathBW + f[:-1]))

Xtrain = np.asarray(Xtrain).flatten()
Xtrain = Xtrain.reshape(-1,w,h,1)
Ytrain = np.asarray(Ytrain).flatten()
Ytrain = Ytrain.reshape((-1,w,h,3))
    
file = open( os.getcwd()+'/Data/test.txt')
testFile = file.readlines()
file.close()

for f in testFile:
    Ytest.append( np.load(pathColor + f[:-1]))
    Xtest.append( np.load(pathBW + f[:-1]))

Xtest = np.asarray(Xtest).flatten()
Xtest = Xtest.reshape(-1,w,h,1)
Ytest = np.asarray(Ytest).flatten()
Ytest = Ytest.reshape((-1,w,h,3))

## Training from 0 -> 200 Epochs, every Step saving the results and model weights

In [ ]:
step = 20
batch = 22
start = 0
end = 1000

for current in range(start,end,step):
    
    
    model.fit(Xtrain, Ytrain, batch_size=batch, validation_data=(Xtest,Ytest), initial_epoch = current, epochs = current+step)
    
    #Create file to dump output to
    os.mkdir(''+os.getcwd()+'/outputs/Model1/Epoch{}'.format(current + step))
    os.mkdir(''+os.getcwd()+'/outputs/Model1/Epoch{}/test'.format(current + step))
    os.mkdir(''+os.getcwd()+'/outputs/Model1/Epoch{}/train'.format(current + step))
 
    # Predict
    output = model.predict(Xtrain[:])
    #Now return the values from 0->1 to 0->255
    output *= 255
    #Convert from f_32 -> u_int8
    output = output.astype('uint8')
    #save train images
    photo_num = 1
    for img in output:
        output = Image.fromarray(img)
        output.save(''+os.getcwd()+'/outputs/Model1/Epoch{}/train/ouput{}.png'.format(current + step,photo_num))
        photo_num += 1
        
    # Predict
    output = model.predict(Xtest[:])
    #Now return the values from 0->1 to 0->255
    output *= 255
    #Convert from f_32 -> u_int8
    output = output.astype('uint8')
    #Save test images
    photo_num = 1
    for img in output:
        z = Image.fromarray(img)
        z.save(''+os.getcwd()+'/outputs/Model1/Epoch{}/test/ouput{}.png'.format(current + step,photo_num))
        photo_num += 1    
        
    #Save model
    model.save( ""+os.getcwd()+"/outputs/Model1/Epoch{}/model1_{}.h5".format(current + step,current + step))